In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(InputLayer((1000, 4)))
model.add(GRU(64))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 64)                13440     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 13,969
Trainable params: 13,969
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

file_path = '/kaggle/input/freezing-of-gait/FOG/freezing-of-gait-exploration-main/freezing-of-gait-exploration-main/sliding_window_dataset'
file_names = [
    'S01R01experiment0', 'S01R02experiment0',
    'S02R01experiment0', 'S02R02experiment0',
    'S03R01experiment0', 'S03R01experiment1', 'S03R02experiment0', 'S03R03experiment0',
    'S04R01experiment0',
    'S05R01experiment0', 'S05R01experiment1', 'S05R01experiment2', 'S05R01experiment3',
    'S06R01experiment0',
]
file_name = [
'S07R01experiment0', 'S07R02experiment0',
    'S08R01experiment0', 'S08R01experiment1', 'S08R01experiment2', 'S08R01experiment3',
    'S09R01experiment0', 'S09R01experiment1'
]

for name in file_names:
    X = np.load(f'{file_path}/{name}_X.npy')
    y = np.load(f'{file_path}/{name}_y.npy')
    y = np.array([[i] for i in y])
    length = X.shape[0]
    test_data_idx_start = int(length * 0.8)
    X_train = X[:test_data_idx_start]
    y_train = y[:test_data_idx_start]
    X_test = X[test_data_idx_start:]
    y_test = y[test_data_idx_start:]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    cp_all = ModelCheckpoint(f'model_{name}/', save_best_only=True)
    stop = EarlyStopping(monitor='val_loss', patience=3)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, callbacks=[cp_all, stop], batch_size=256)


(14560, 1000, 4) (14560, 1) (3640, 1000, 4) (3640, 1)
Epoch 1/15
57/57 [==============================] - 69s 1s/step - loss: 0.6732 - accuracy: 0.5859 - val_loss: 0.5671 - val_accuracy: 0.7179
Epoch 2/15
57/57 [==============================] - 67s 1s/step - loss: 0.3096 - accuracy: 0.8973 - val_loss: 0.2829 - val_accuracy: 0.9618
Epoch 3/15
57/57 [==============================] - 69s 1s/step - loss: 0.1478 - accuracy: 0.9859 - val_loss: 0.1211 - val_accuracy: 0.9942
Epoch 4/15
57/57 [==============================] - 68s 1s/step - loss: 0.0676 - accuracy: 0.9992 - val_loss: 0.0748 - val_accuracy: 0.9970
Epoch 5/15
57/57 [==============================] - 67s 1s/step - loss: 0.0456 - accuracy: 0.9997 - val_loss: 0.0519 - val_accuracy: 0.9978
Epoch 6/15
57/57 [==============================] - 67s 1s/step - loss: 0.0336 - accuracy: 0.9997 - val_loss: 0.0408 - val_accuracy: 0.9978
Epoch 7/15
57/57 [==============================] - 68s 1s/step - loss: 0.0261 - accuracy: 0.9999 - val_lo

In [ ]:
for name in file_name:
    X1 = np.load(f'{file_path}/{name}_X.npy')
    length = X1.shape[0]
    y1 = np.load(f'{file_path}/{name}_y.npy')
    y1 = np.array([[i] for i in y1])
    test_data_idx_start = int(length)
    X1_test = X1[:test_data_idx_start]
    y1_test = y1[:test_data_idx_start]

In [ ]:
yhat = model.predict(X1_test)# metrics# metrics
from sklearn import metrics
TN, FP, FN, TP = metrics.confusion_matrix(y1_test, np.array([0 if y < .5 else 1 for y in yhat])).ravel()


print('Sensitivity, hit rate, recall, or true positive rate: ')
TPR = TP/(TP+FN)
print(TPR)
print('Specificity or true negative rate')
TNR = TN/(TN+FP) 
print(TNR)
print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)